In [90]:
#!pip install elasticsearch-dsl
#!pip install certifi
#!pip install ujson 
import requests
import ujson as json

url = 'http://ec2-52-91-189-234.compute-1.amazonaws.com:9200'
ES_HOST = {"host" : "ec2-52-91-189-234.compute-1.amazonaws.com", "port" : 9200}


    100% |████████████████████████████████| 194kB 222kB/s ta 0:00:01
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/9e/9b/d0/df92653bb5b2664c15d8ee5b99e3f2eb08a034444db8922b2f
Successfully built ujson


In [16]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
client = Elasticsearch()

In [91]:
res = requests.get(url +'/books/book/_mapping?pretty')
#print(res.json)

import json
j = json.loads(res.text)
print(j['books']['mappings']['book'])


for i in j['books']['mappings']['book']:
    print("a")


import pandas as pd
from pandas.io.json import json_normalize

#df = pd.DataFrame(j['books']['mappings']['book'])






df = json_normalize(j['books']['mappings']['book'])
df.head()

{'properties': {'description': {'type': 'text'}, 'price': {'scaling_factor': 100.0, 'type': 'scaled_float'}, 'pub_date': {'format': 'yyyy-MM-dd', 'type': 'date'}, 'title': {'type': 'text'}, 'tags': {'type': 'text'}, 'language': {'type': 'keyword'}, 'isbn': {'type': 'integer'}, 'author': {'type': 'text'}}}
a


,properties.author.type,properties.description.type,properties.isbn.type,properties.language.type,properties.price.scaling_factor,properties.price.type,properties.pub_date.format,properties.pub_date.type,properties.tags.type,properties.title.type
0,text,text,integer,keyword,100.0,scaled_float,yyyy-MM-dd,date,text,text


In [19]:
FILE_URL = "http://apps.sloanahrens.com/qbox-blog-resources/kaggle-titanic-data/test.csv"
INDEX_NAME = 'titanic'
TYPE_NAME = 'passenger'
ID_FIELD = 'passengerid'

In [20]:
from datetime import datetime
from elasticsearch_dsl import DocType, Date, Integer, Keyword, Text
from elasticsearch_dsl.connections import connections
# Define a default Elasticsearch client
es = Elasticsearch(url)
#es.create(index="test", doc_type="articles", body={"content": "One more fox"})
es.get(id='78', index="test")



{'_id': '78',
 '_index': 'test',
 '_source': {'content': 'One more fox'},
 '_type': 'articles',
 '_version': 1,
 'found': True}

In [21]:
doc = {
    'author': 'kimchy',
    'text': 'Nothing lasts',
    'timestamp': datetime.now(),
}
res = es.index(index="test-index", doc_type='tweet', id=1, body=doc)
print(res['created'])

False


In [63]:
s = Search(using=es, index="test-index") \
    .filter("term", category="search") \
    .query("match", title="Elastic")   \
    .exclude("match", description="beta")


s.aggs.bucket('per_tag', 'terms', field='tags') \
    .metric('max_lines', 'max', field='lines')

response = s.execute()
print
for hit in response:
    print(hit.meta.score, hit.title)

for tag in response.aggregations.per_tag.buckets:
    print(tag.key, tag.max_lines.value)

In [33]:
res = es.get(index="test-index", doc_type='tweet', id=1)
print(res)

{'_type': 'tweet', '_index': 'test-index', '_source': {'text': 'Nothing lasts', 'timestamp': '2017-05-23T15:46:58.908242', 'author': 'kimchy'}, 'found': True, '_id': '1', '_version': 8}


In [24]:
doc = {
        "query": {
                    "match_all": {}
                }
        }
es.search(index="test-index", body=doc)




{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '1',
    '_index': 'test-index',
    '_score': 1.0,
    '_source': {'author': 'kimchy',
     'text': 'Nothing lasts',
     'timestamp': '2017-05-23T15:46:58.908242'},
    '_type': 'tweet'}],
  'max_score': 1.0,
  'total': 1},
 'timed_out': False,
 'took': 0}

In [85]:
res  = es.search(index="test-index", body=doc)

for hit in res['hits']['hits']:
    print(hit)
    
df = json_normalize(res['hits']['hits'])
df.head()

{'_type': 'tweet', '_index': 'test-index', '_id': '1', '_score': 1.0, '_source': {'text': 'Nothing lasts', 'timestamp': '2017-05-23T15:46:58.908242', 'author': 'kimchy'}}


,_id,_index,_score,_source.author,_source.text,_source.timestamp,_type
0,1,test-index,1.0,kimchy,Nothing lasts,2017-05-23T15:46:58.908242,tweet


In [32]:
#/test/mytype/_mapping

res = es.get_source(index="test-index", doc_type='tweet', id=1)
print(res)
res = es.

{'text': 'Nothing lasts', 'timestamp': '2017-05-23T15:46:58.908242', 'author': 'kimchy'}


In [11]:
es.indices.create(index='test-index1', ignore=[400,404])

{'error': {'index': 'test-index1',
  'index_uuid': 'fY3utQb5T-a35zTIEjLnPg',
  'reason': 'index [test-index1/fY3utQb5T-a35zTIEjLnPg] already exists',
  'root_cause': [{'index': 'test-index1',
    'index_uuid': 'fY3utQb5T-a35zTIEjLnPg',
    'reason': 'index [test-index1/fY3utQb5T-a35zTIEjLnPg] already exists',
    'type': 'index_already_exists_exception'}],
  'type': 'index_already_exists_exception'},
 'status': 400}

In [12]:
es.search(index='test-index', filter_path=['hits.hits', 'hits.hits._source'])

{'hits': {'hits': [{'_id': '1',
    '_index': 'test-index',
    '_score': 1.0,
    '_source': {'author': 'kimchy',
     'text': 'Nothing lasts',
     'timestamp': '2017-05-23T13:11:43.944443'},
    '_type': 'tweet'}]}}

In [13]:
es.search(index='test-index', filter_path=['hits.hits._source'])

{'hits': {'hits': [{'_source': {'author': 'kimchy',
     'text': 'Nothing lasts',
     'timestamp': '2017-05-23T13:11:43.944443'}}]}}

In [14]:
es.search(index='test-index')

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '1',
    '_index': 'test-index',
    '_score': 1.0,
    '_source': {'author': 'kimchy',
     'text': 'Nothing lasts',
     'timestamp': '2017-05-23T13:11:43.944443'},
    '_type': 'tweet'}],
  'max_score': 1.0,
  'total': 1},
 'timed_out': False,
 'took': 0}